# Chapter 14: Events

Before you start, please comment out the `validate_product_name()` method of the `Product` model first:

```python
class Product(Base, repr=False):  # type: ignore
    __tablename__ = "product"

    product_id: Mapped[int_pk] = mapped_column(init=False)

    ...

    # @validates("product_name")
    # def validate_product_name(self, key, value: str):
    #     print("key:", key, "value:", value)
    #     return value.title()
```

Load models from part 3:

In [1]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory of the current directory
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Get the sibling directory's path
sibling_dir = os.path.join(parent_dir, 'part3')

# Add the sibling directory to the Python path
sys.path.append(sibling_dir)

In [2]:
from models import *

## Registering Event Listeners

In [3]:
from sqlalchemy import select
from sqlalchemy.event import listen, listens_for, remove
from sqlalchemy.orm import Session

Event listener registration using the `listen()` function:

In [4]:
def db_on_connect1(dbapi_connection, connection_record):
    print(f"Connected 1: {dbapi_connection}")

listen(engine, "connect", db_on_connect1)

Event listener registration using the `listens_for()` decorator:

In [5]:
@listens_for(engine, "connect")
def db_on_connect2(dbapi_connection, connection_record):
    print(f"Connected 2: {dbapi_connection}")

Triggering event listeners for "connect" by listing products:

In [6]:
def list_products(session: Session):
    """List all products."""
    products = session.scalars(select(Product))
    print("Listing products...")
    for product in products:
        print(product)

In [7]:
with SessionMaker() as session:
    print(">> Session starts!")
    list_products(session)
    print(">> Session ends!")

>> Session starts!
Connected 1: <sqlite3.Connection object at 0x7f1cfa8f5640>
Connected 2: <sqlite3.Connection object at 0x7f1cfa8f5640>
2024-03-30 15:13:16,785 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-30 15:13:16,803 INFO sqlalchemy.engine.Engine SELECT product.product_id, product.product_name, product.unit_price, product.units_in_stock, product.type 
FROM product
2024-03-30 15:13:16,803 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
Listing products...
>> Session ends!
2024-03-30 15:13:16,805 INFO sqlalchemy.engine.Engine ROLLBACK


## ORM events:

Defining a listener for mapper events:

In [8]:
@listens_for(Product, 'before_insert')
@listens_for(Product, 'before_update')
def before_insert_update(mapper, connection, target: Product):
    print(f"Input product name: {target.product_name}")

    # converting product name to title case
    target.product_name = target.product_name.title()

Data for product record:

In [9]:
product_data = {
    "product_name": "phone screen protector",
    "unit_price": 9.50,
    "units_in_stock": 10,
    "type": ProductType.ACCESSORY,
}

Functions for triggering the product event listener:

In [10]:
def insert_product(session: Session, data):
    product = Product(**data)
    session.add(product)
    session.commit()

    return product.product_id

In [11]:
def update_product(session: Session, product_id: int, new_product_name: str):
    product = session.get(Product, product_id)
    if product is not None:
        product.product_name = new_product_name
        session.commit()

Insert product:

In [12]:
session = SessionMaker()

In [13]:
product_id = insert_product(session, product_data)

2024-03-30 15:13:30,632 INFO sqlalchemy.engine.Engine BEGIN (implicit)
Input product name: phone screen protector
2024-03-30 15:13:30,635 INFO sqlalchemy.engine.Engine INSERT INTO product (product_name, unit_price, units_in_stock, type) VALUES (?, ?, ?, ?)
2024-03-30 15:13:30,636 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ('Phone Screen Protector', 9.5, 10, 'ACCESSORY')
2024-03-30 15:13:30,637 INFO sqlalchemy.engine.Engine COMMIT
2024-03-30 15:13:30,647 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-30 15:13:30,649 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-30 15:13:30,649 INFO sqlalchemy.engine.Engine [generated in 0.00043s] (1,)


In [14]:
print("New product added:", session.get(Product, product_id))

2024-03-30 15:13:35,805 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-30 15:13:35,807 INFO sqlalchemy.engine.Engine [generated in 0.00144s] (1,)
New product added: Product(product_id=1, product_name='Phone Screen Protector', unit_price=9.50, units_in_stock=10, type='accessory')


Update product:

In [15]:
update_product(session, product_id, "phone screen protector 2023")

2024-03-30 15:13:39,144 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-30 15:13:39,145 INFO sqlalchemy.engine.Engine [cached since 3.34s ago] (1,)
Input product name: phone screen protector 2023
2024-03-30 15:13:39,148 INFO sqlalchemy.engine.Engine UPDATE product SET product_name=? WHERE product.product_id = ?
2024-03-30 15:13:39,149 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ('Phone Screen Protector 2023', 1)
2024-03-30 15:13:39,150 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
print("Product updated to:", session.get(Product, product_id))

2024-03-30 15:13:41,841 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-30 15:13:41,841 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-30 15:13:41,842 INFO sqlalchemy.engine.Engine [cached since 6.037s ago] (1,)
Product updated to: Product(product_id=1, product_name='Phone Screen Protector 2023', unit_price=9.50, units_in_stock=10, type='accessory')


Remove event listeners:

In [17]:
remove(Product, 'before_insert', before_insert_update)
remove(Product, 'before_update', before_insert_update)

In [18]:
product_data["product_name"] = 'test'

Check that the event listeners are actually removed:

In [19]:
product_id = insert_product(session, product_data)

2024-03-30 15:13:47,683 INFO sqlalchemy.engine.Engine INSERT INTO product (product_name, unit_price, units_in_stock, type) VALUES (?, ?, ?, ?)
2024-03-30 15:13:47,684 INFO sqlalchemy.engine.Engine [cached since 17.05s ago] ('test', 9.5, 10, 'ACCESSORY')
2024-03-30 15:13:47,685 INFO sqlalchemy.engine.Engine COMMIT
2024-03-30 15:13:47,698 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-30 15:13:47,699 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-30 15:13:47,699 INFO sqlalchemy.engine.Engine [cached since 17.05s ago] (2,)


In [20]:
print("New product added:", session.get(Product, product_id))

2024-03-30 15:13:50,076 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-30 15:13:50,076 INFO sqlalchemy.engine.Engine [cached since 14.27s ago] (2,)
New product added: Product(product_id=2, product_name='test', unit_price=9.50, units_in_stock=10, type='accessory')


In [21]:
update_product(session, product_id, "test updated")

2024-03-30 15:13:52,162 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-30 15:13:52,163 INFO sqlalchemy.engine.Engine [cached since 16.36s ago] (2,)
2024-03-30 15:13:52,165 INFO sqlalchemy.engine.Engine UPDATE product SET product_name=? WHERE product.product_id = ?
2024-03-30 15:13:52,165 INFO sqlalchemy.engine.Engine [cached since 13.02s ago] ('test updated', 2)
2024-03-30 15:13:52,166 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
print("Product updated to:", session.get(Product, product_id))

2024-03-30 15:13:54,264 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-30 15:13:54,265 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-30 15:13:54,266 INFO sqlalchemy.engine.Engine [cached since 18.46s ago] (2,)
Product updated to: Product(product_id=2, product_name='test updated', unit_price=9.50, units_in_stock=10, type='accessory')


In [23]:
session.close()

2024-03-30 15:13:58,507 INFO sqlalchemy.engine.Engine ROLLBACK


Uncomment the `validate_product_name()` method:

```python
class Product(Base, repr=False):
    ...
    @validates("product_name")
    def validate_product_name(self, key, value: str): ❶
        return value.title()
```


Restart the kernel!!!

In [1]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory of the current directory
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Get the sibling directory's path
sibling_dir = os.path.join(parent_dir, 'part3')

# Add the sibling directory to the Python path
sys.path.append(sibling_dir)

In [2]:
from models import *

In [3]:
session = SessionMaker()

In [4]:
from sqlalchemy import select
from sqlalchemy.event import listen, listens_for, remove
from sqlalchemy.orm import Session

Test the listener with product insertion:

In [5]:
def insert_product(session: Session, data):
    product = Product(**data)
    session.add(product)
    session.commit()

    return product.product_id

In [6]:
product_data = {
    "product_name": "phone screen protector",
    "unit_price": 9.50,
    "units_in_stock": 10,
    "type": ProductType.ACCESSORY,
}

In [7]:
product_id_2 = insert_product(session, product_data)

key: product_name value: phone screen protector
2024-03-30 15:14:19,538 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-30 15:14:19,541 INFO sqlalchemy.engine.Engine INSERT INTO product (product_name, unit_price, units_in_stock, type) VALUES (?, ?, ?, ?)
2024-03-30 15:14:19,542 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ('Phone Screen Protector', 9.5, 10, 'ACCESSORY')
2024-03-30 15:14:19,543 INFO sqlalchemy.engine.Engine COMMIT
2024-03-30 15:14:19,557 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-30 15:14:19,562 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-30 15:14:19,562 INFO sqlalchemy.engine.Engine [generated in 0.00092s] (3,)


In [8]:
print("New product added:", session.get(Product, product_id_2))

2024-03-30 15:14:22,744 INFO sqlalchemy.engine.Engine SELECT product.product_id AS product_product_id, product.product_name AS product_product_name, product.unit_price AS product_unit_price, product.units_in_stock AS product_units_in_stock, product.type AS product_type 
FROM product 
WHERE product.product_id = ?
2024-03-30 15:14:22,745 INFO sqlalchemy.engine.Engine [generated in 0.00174s] (3,)
New product added: Product(product_id=3, product_name='Phone Screen Protector', unit_price=9.50, units_in_stock=10, type='accessory')


Email Validation Function:

Add the `validate_email()` method to the `Customer` model:

```python
class Customer(Base):
    ...

    @validates("email")
    def validate_email(self, key, value):
        if not is_email_valid(value):
            raise ValueError("Email validation failed!")
        return value
```

Testing the email validator with customer data:

In [9]:
customer_data = {
    "first_name": "Alex",
    "last_name": "Smith",
    "address": "618 Oak Lane, CA",
    "email": "alex_smith@test.com",
}

In [10]:
def insert_customer(session: Session, data):
    customer = Customer(**data)
    session.add(customer)
    session.commit()

    return customer.customer_id

In [11]:
customer_id = insert_customer(session, customer_data)

2024-03-30 15:14:32,607 INFO sqlalchemy.engine.Engine INSERT INTO customer (first_name, last_name, address, email) VALUES (?, ?, ?, ?)
2024-03-30 15:14:32,608 INFO sqlalchemy.engine.Engine [generated in 0.00093s] ('Alex', 'Smith', '618 Oak Lane, CA', 'alex_smith@test.com')
2024-03-30 15:14:32,609 INFO sqlalchemy.engine.Engine COMMIT
2024-03-30 15:14:32,625 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-30 15:14:32,626 INFO sqlalchemy.engine.Engine SELECT customer.customer_id AS customer_customer_id, customer.first_name AS customer_first_name, customer.last_name AS customer_last_name, customer.email AS customer_email 
FROM customer 
WHERE customer.customer_id = ?
2024-03-30 15:14:32,627 INFO sqlalchemy.engine.Engine [generated in 0.00054s] (1,)


In [12]:
print("Customer inserted:", session.get(Customer, customer_id))

2024-03-30 15:14:34,915 INFO sqlalchemy.engine.Engine SELECT customer.customer_id AS customer_customer_id, customer.first_name AS customer_first_name, customer.last_name AS customer_last_name, customer.email AS customer_email 
FROM customer 
WHERE customer.customer_id = ?
2024-03-30 15:14:34,916 INFO sqlalchemy.engine.Engine [generated in 0.00140s] (1,)
Customer inserted: 2024-03-30 15:14:34,919 INFO sqlalchemy.engine.Engine SELECT customer.address AS customer_address 
FROM customer 
WHERE customer.customer_id = ?
2024-03-30 15:14:34,920 INFO sqlalchemy.engine.Engine [generated in 0.00066s] (1,)
Customer(customer_id=1, first_name='Alex', last_name='Smith', address='618 Oak Lane, CA', email='alex_smith@test.com')


In [13]:
try:
    customer_data["email"] = "not_valid"
    customer_id = insert_customer(session, customer_data)
except ValueError as e:
    print("A value error occurred:", e)

A value error occurred: Email validation failed!
